In [15]:
from config import *
from test_bad_word import *

import time
import numpy as np
import pandas as pd
#pd.options.display.max_columns = None
#pd.options.display.mpl_style = 'default'

import re
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

In [16]:
df_train_1 = pd.read_csv(DATA_DIR + '/train.csv', encoding="ISO-8859-1")
df_train_2 = pd.read_csv(DATA_DIR + '/test_with_solutions.csv', encoding="ISO-8859-1")
df_train = pd.concat((df_train_1, df_train_2), axis=0, ignore_index=True)

In [17]:
def parser(text):
    # encode editor
    text = re.sub(r'\\\\', r'\\', text)
    text = re.sub(r'\\\\', r'\\', text)
    # remove non-break symbol
    text = re.sub(r'\\x\w{2,2}', ' ', text)
    text = re.sub(r'\\u\w{4,4}', ' ', text)
    # remove change line symbol
    text = text.replace('\\n\\n','. ')
    text = text.replace('\\n','. ')
    
    # Grammer
    text = text.replace("'ll", " will")
    text = text.replace("'LL", " WILL")
    text = text.replace("'d", " would")
    text = text.replace("'D", " WOULD")
    text = text.replace("n't", "nt")
    text = text.replace("N'T", "NT")
        
    # Format whitespaces
    text = text.replace('"', ' ')
    text = text.replace('\'', ' ')
    text = text.replace('_', ' ')
    text = text.replace('-', ' ')
    text = text.replace('\n', ' ')
    text = re.sub(' +',' ', text)
    
    # Convert words consisted by more than 3 symbols to special curse token "_CR_"
    text = re.sub('\s([#%&\*\$]{2,})\s', ' _CR_ ', text)
    # Convert email adresses to email token '_email_'
    text = re.sub('[\w\-][\w\-\.]+@[\w\-][\w\-\.]+[a-zA-Z]{1,4}', '_email_', text)
    # Convert urls to url token
    text = re.sub('\w+:\/\/\S+', r'_url_', text)
    # Convert hashtags to hashtage token
    text = re.sub('@\w\w+:?',r'_hashtag_', text)
    # convert word with repeated charactors into normal form, eg: coooooool --> cool
    text = re.sub(r'([a-zA-Z])\1\1+(\w*)', r'\1\1\2', text)
    
    # Unify you are/YOU ARE
    text = re.sub("[Yy]ou'? ?a?re ", "you are ", text)
    text = re.sub("YOU'? ?A?RE ", "YOU ARE ", text)
    
    # sepearate punctuation with word
    text = re.sub(r'(\S)([?.,!]+)', r'\1 \2 ', text)
    
    return text

df_train['Comment']=df_train['Comment'].map(lambda x:parser(x))

In [18]:
f = open(DATA_DIR + '/badwords.txt', "r")
bwMap = dict()
for line in f:
    sp = line.strip().lower().split(",")
    if len(sp) == 2:
        bwMap[sp[0].strip()] = sp[1].strip()
    
def badword_replacer(text):   
    for key, value in bwMap.items():
        kpad = " " + key + " "
        vpad = " " + value + " "
        text = text.replace(kpad, vpad) 
    return text

df_train['Comment']=df_train['Comment'].map(lambda x:badword_replacer(x))

In [24]:
bwMap

{'8ss': 'ass',
 '@$$': 'ass',
 '@ss': 'ass',
 'a s s': 'ass',
 'a$$': 'ass',
 'a-hole': 'asshole',
 'ahole': 'asshole',
 'ash0le': 'asshole',
 'ash0les': 'asshole',
 'asholes': 'asshole',
 'ass monkey': 'asshole',
 'ass-hole': 'asshole',
 'assface': 'asshole',
 'assh0le': 'asshole',
 'assh0lez': 'asshole',
 'assholz': 'asshole',
 'asswipe': 'asshole',
 'azz': 'ass',
 'azzhole': 'asshole',
 'b!+ch': 'bitch',
 'b!tch': 'bitch',
 'b*tch': 'bitch',
 'b00bs': 'boobs',
 'b17ch': 'bitch',
 'b1tch': 'bitch',
 'bassterds': 'basterd',
 'bastardz': 'basterd',
 'basterds': 'basterd',
 'basterdz': 'basterd',
 'bi+ch': 'bitch',
 'bi7ch': 'bitch',
 'biatch': 'bitch',
 'bitches': 'bitch',
 'blow job': 'blowjob',
 'butthole': 'asshole',
 'buttwipe': 'asshole',
 'c0ck': 'cock',
 'c0cks': 'cock',
 'c0k': 'cock',
 'cawk': 'cock',
 'cawks': 'cock',
 'cnts': 'cunt',
 'cntz': 'cunt',
 'cock-head': 'cock',
 'cock-sucker': 'cocksucker',
 'cockhead': 'cock',
 'cuntz': 'cunt',
 'd!ck': 'dick',
 'd*ck': 'dick',
 

In [23]:
df_train['Comment']

0                                   You fuck your dad .  
1        i really dont understand your point .  It see...
2        A majority of Canadians can and has been wron...
3        listen if you dont wanna get married to a man...
4        C c b n xu ng ng bi u t nh 2011 c n ho kh ng ...
5        _hashtag_ OK ,  but I would hope they would s...
6                          Yeah and where are you now ?  
7        shut the fuck up .  you and the rest of your ...
8        Either you are fake or extremely stupid ... m...
9        That you are an idiot who understands neither...
10       _hashtag_ dont wish him injury but it happene...
11       Be careful , Jimbo . OG has a fork with your ...
12             _hashtag_ Or they just dont pay attention 
13       Hmm .  Perhaps some who are too pig faced to ...
14       _hashtag_ Huh ?  Her income was $21 , 912 .  ...
15       FOR SOME REASON U SOUND RETARDED .  LOL .  DA...
16       You with the racist screen name .  you are a ...
17       Oh ! 

In [19]:
df_train['length']=df_train['Comment'].map(lambda x:len(x.split()))

bad_word_1  = [line.rstrip('\n') for line in open('google_bad_word.txt')]
#bad_word_2  = [line.rstrip('\n') for line in open('handcrafted_badword.txt')]
bad_word= set(bad_word_1  + test_bad_word) 

df_train['bad word count']=df_train['Comment'].map(lambda x:sum([word.lower() in bad_word for word in x.split()]))
df_train['bad word ratio']=df_train['bad word count']/df_train['length']

In [20]:
df_non = df_train[df_train['Insult']==1][df_train['bad word count']==0]

C:\Users\zcakz\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [21]:
len(df_non)

242

In [22]:
df_non.to_csv("df_insult_test_without_badword.csv", index=False,encoding = 'utf-8')